In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression, Ridge
from sklearn.metrics import mean_squared_error, r2_score
import joblib
import seaborn as sns
import matplotlib.pyplot as plt

# 1. Load Dataset

In [ ]:
df = pd.read_csv("boston.csv")

# 2. Data Preprocessing

In [ ]:
X = df.drop("medv", axis=1)    # medv = target column
y = df["medv"]

In [ ]:
# Train-test split
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

# Feature scaling
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Save the scaler
joblib.dump(scaler, "scaler.pkl")

# 3. Model Training

In [ ]:
# Linear Regression
lr = LinearRegression()
lr.fit(X_train_scaled, y_train)

# Ridge Regression (alpha tuned manually, you can grid search)
ridge = Ridge(alpha=1.0)
ridge.fit(X_train_scaled, y_train)

# 4. Model Evaluation

In [ ]:
def evaluate(model, X_test_scaled, y_test, title):
    preds = model.predict(X_test_scaled)
    mse = mean_squared_error(y_test, preds)
    r2 = r2_score(y_test, preds)
    print(f"{title} -> MSE: {mse:.3f}, R2: {r2:.3f}")
    return mse, r2

print("\nModel Performance:")
lr_mse, lr_r2 = evaluate(lr, X_test_scaled, y_test, "Linear Regression")
ridge_mse, ridge_r2 = evaluate(ridge, X_test_scaled, y_test, "Ridge Regression")

# Pick the best model
if ridge_r2 > lr_r2:
    best_model = ridge
    print("\nRidge Regression selected as best model.")
else:
    best_model = lr
    print("\nLinear Regression selected as best model.")

# 5. Save Best Model

In [ ]:
joblib.dump(best_model, "best_model.pkl")
print("Model saved as best_model.pkl")

# 6. Data Visualization

In [ ]:
plt.figure(figsize=(10, 8))
sns.heatmap(df.corr(), annot=False, cmap="viridis")
plt.title("Correlation Heatmap")
plt.savefig("heatmap.png")

# Prediction vs Actual
preds = best_model.predict(X_test_scaled)
plt.figure(figsize=(8, 6))
plt.scatter(y_test, preds)
plt.xlabel("Actual Prices")
plt.ylabel("Predicted Prices")
plt.title("Prediction vs Actual")
plt.savefig("prediction_vs_actual.png")

print("Visualizations saved.")